In this notebook we want to build a dataframe that consists of each kicker's 50th kick. 50 because that is what Going For Three used.

In [3]:
import pandas as pd
import numpy as np


def check_sorted_descending(values):
    sorted_ = list(np.sort(values))
    sorted_.reverse()
    return all([i==j for i,j in zip(values,sorted_)])

Get a dataframe consisting of all field goal attempts in the dataset.

In [74]:
df_kicks = pd.read_csv('../data/nfl_00-19/FGXP.csv')
df_plays = pd.read_csv('../data/nfl_00-19/PLAY.csv')
df_game = pd.read_csv('../data/nfl_00-19/GAME.csv')
df_kicks = df_kicks.merge(df_plays, on='pid', suffixes=('',''))
df_kicks = df_kicks.merge(df_game, on='gid', suffixes=('',''))
df_kicks = df_kicks.loc[:,['pid','gid','fgxp','fkicker','dist','good']]
df_fgs = df_kicks.loc[df_kicks['fgxp']=='FG',:]

In [78]:
len(df_fgs)

44758

Now filter out attempts by kickers that have had less than 50 attempts in total.

In [114]:
df_exp = df_fgs.groupby('fkicker').filter(lambda x: len(x)>=50)

For the kickers remaining: group their kicks, order by time of attempt (oldest first), take top 50, re-order (latest first), and take the top row (50th kick)

In [122]:
fiftieths = pd.DataFrame(columns=df_exp.columns)

for i,group in df_exp.groupby('fkicker'):
    group = group.sort_values('pid', ascending=True).head(50).sort_values('pid', ascending=False)
    assert check_sorted_descending(group['gid']), 'Games not in order.'
    fiftieths = fiftieths.append(group.iloc[0,:])

Save the resulting dataframe to csv

In [127]:
fiftieths.drop(['fgxp'], inplace=True, axis=1)
fiftieths.to_csv('FIFTY.csv', index=False)

In [130]:
fiftieths.head()

,pid,gid,fkicker,dist,good
28030,550548,3379,AH-2600,43,1
41808,815802,4975,AR-1450,57,1
3171,65028,401,AV-0400,33,1
7362,148410,910,BC-2300,36,1
8578,172642,1059,BC-3000,27,1
